In [ ]:
# fix for relative imports when running as a script
import sys
sys.path.append("..")

In [ ]:
# import modules from the bubble package
from bubble.loader import load_stock_data
from bubble.indicators import (
    calculate_returns,
    calculate_index_returns,
    build_equal_weight_index,
    calculate_rolling_volatility,
    calculate_rolling_correlation,
    calculate_rolling_sharpe,
    calculate_drawdown,
    calculate_max_drawdown,
)
from bubble.visualizations import (
    plot_correlation_matrix,
    plot_rolling_correlation,
    plot_rolling_sharpe,
    plot_index_vs_benchmark,
    plot_drawdown,
    plot_return_distribution,
)

In [ ]:
# define the list of stock tickers
tickers = [
    "NVDA", 
    "MSFT", 
    "GOOGL", 
    "META", 
    "AMZN",
    "TSM", 
    "AMD", 
    "AVGO", 
    "ASML", 
    "BOTZ", 
    "ARKQ", 
    "AIQ"
]

# define benchmark tickers
benchmark_tickers = [
    "QQQ", 
    "SPY", 
    "SOXX"
]

In [ ]:
# load stock data for AI-related companies
ai_stock_data = load_stock_data(tickers)
ai_stock_data.tail(3)

In [ ]:
# print summary information about the loaded data
print("✅ AI stock data loaded")
print(f"tickers: {', '.join(map(str, ai_stock_data.columns))}")
print(
    f"data range: {ai_stock_data.index.min().date()} → "
    f"{ai_stock_data.index.max().date()}"
)
print(f"number of rows: {len(ai_stock_data)}")

In [ ]:
# load stock data for benchmark indices
benchmark_stock_data = load_stock_data(benchmark_tickers)
benchmark_stock_data.tail(3)

In [ ]:
# print summary information about the loaded data
print("✅ Benchmark stock data loaded")
print(f"tickers: {', '.join(map(str, benchmark_stock_data.columns))}")
print(
    f"data range: {benchmark_stock_data.index.min().date()} → "
    f"{benchmark_stock_data.index.max().date()}"
)
print(f"number of rows: {len(benchmark_stock_data)}")

In [ ]:
# calculate log returns for AI stocks
ai_log_returns = calculate_returns(ai_stock_data, log_returns=True)
ai_log_returns.tail(3)

In [ ]:
# calculate rolling volatility correlation for the 60-day window
corr_matrix = calculate_rolling_correlation(ai_log_returns, window=60)
corr_matrix.tail(3)

In [ ]:
# calculate rolling volatility for the 60-day window
rolling_vol = calculate_rolling_volatility(ai_log_returns, window=60)
rolling_vol.tail(3)

In [ ]:
# build equal-weighted index for AI stocks
ai_index = build_equal_weight_index(ai_stock_data)
ai_index_returns = calculate_index_returns(ai_index, log_returns=True)

In [ ]:
# initialize dictionaries for benchmark indices and returns
bench_indices = {}
bench_returns = {}

# normalize benchmark stock data
bench_normalized = benchmark_stock_data / benchmark_stock_data.iloc[0]

# calculate benchmark indices and returns
for ticker in benchmark_tickers:
    idx = benchmark_stock_data[ticker]
    bench_indices[ticker] = idx
    bench_returns[ticker] = calculate_index_returns(idx, log_returns=True)